In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [69]:
train = pd.read_csv(r'data/X_train.csv')
target = pd.read_csv(r'data/y_train.csv')

In [70]:
train.drop(['row_id','measurement_number'], axis=1, inplace=True)
train.set_index('series_id',inplace=True)

In [53]:
train

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
series_id,,,,,,,,,,
0,-0.75853,-0.63435,-0.104880,-0.105970,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
0,-0.75853,-0.63434,-0.104900,-0.106000,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
0,-0.75853,-0.63435,-0.104920,-0.105970,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
0,-0.75852,-0.63436,-0.104950,-0.105970,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
0,-0.75852,-0.63435,-0.104950,-0.105960,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410
...,...,...,...,...,...,...,...,...,...,...
3809,0.62871,-0.76878,-0.084391,0.081093,0.003167,0.093760,-0.142740,3.27180,2.0115,-9.0063
3809,0.62884,-0.76868,-0.084365,0.081099,0.014994,0.032637,-0.132380,4.42750,3.0696,-8.1257
3809,0.62891,-0.76861,-0.084345,0.081178,-0.031184,-0.003961,-0.138940,2.70480,4.2622,-8.1443


In [54]:
from keras.utils import normalize
from sklearn.preprocessing import scale
train[train.columns] = normalize(train)
train

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
series_id,,,,,,,,,,
0,-0.075432,-0.063083,-0.010430,-0.010538,0.010705,0.001746,0.000076,-0.074441,0.209132,-0.969905
0,-0.079086,-0.066138,-0.010937,-0.011052,0.007074,0.003122,0.000353,0.035444,0.157061,-0.981400
0,-0.084933,-0.071029,-0.011748,-0.011866,0.000815,0.003240,-0.000669,-0.029593,0.178280,-0.977134
0,-0.074265,-0.062108,-0.010275,-0.010375,-0.001278,0.001904,-0.000879,0.041791,0.107630,-0.988472
0,-0.071536,-0.059825,-0.009898,-0.009993,0.000484,0.000722,0.000495,-0.048069,0.138531,-0.984684
...,...,...,...,...,...,...,...,...,...,...
3809,0.063871,-0.078101,-0.008573,0.008238,0.000322,0.009525,-0.014501,0.332384,0.204349,-0.914954
3809,0.064157,-0.078424,-0.008607,0.008274,0.001530,0.003330,-0.013506,0.451713,0.313174,-0.829019
3809,0.065274,-0.079773,-0.008754,0.008425,-0.003237,-0.000411,-0.014420,0.280728,0.442369,-0.845287


In [71]:
data = np.array(list(train.groupby('series_id').apply(pd.DataFrame.as_matrix)))

In [81]:
data.shape

(3810, 128, 10)

In [73]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(target.surface.values.reshape(-1, 1))
t = ohe.transform(target.surface.values.reshape(-1, 1))

In [94]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, LSTM
model = Sequential()
model.add(LSTM(32, input_shape = (128,10)))
model.add(Dense(9, activation='softmax'))
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5504      
_________________________________________________________________
dense_15 (Dense)             (None, 9)                 297       
Total params: 5,801
Trainable params: 5,801
Non-trainable params: 0
_________________________________________________________________


In [78]:
t.todense()

matrix([[0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [95]:
np.random.seed(42)
history = model.fit(data, t,
                    batch_size=128,
                    epochs=300,
                    validation_split=0.2)

Train on 3048 samples, validate on 762 samples
Epoch 1/300
3048/3048 [==============================] - 3s 1ms/step - loss: 2.1275 - acc: 0.1890 - val_loss: 2.0194 - val_acc: 0.1745
Epoch 2/300
3048/3048 [==============================] - 2s 799us/step - loss: 2.0256 - acc: 0.2365 - val_loss: 1.9922 - val_acc: 0.2126
Epoch 3/300
3048/3048 [==============================] - 2s 771us/step - loss: 1.9752 - acc: 0.2835 - val_loss: 1.9670 - val_acc: 0.2349
Epoch 4/300
3048/3048 [==============================] - 2s 799us/step - loss: 1.9166 - acc: 0.3097 - val_loss: 1.9397 - val_acc: 0.2520
Epoch 5/300
3048/3048 [==============================] - 2s 771us/step - loss: 1.8469 - acc: 0.3182 - val_loss: 1.8525 - val_acc: 0.2756
Epoch 6/300
3048/3048 [==============================] - 2s 766us/step - loss: 1.7722 - acc: 0.3428 - val_loss: 1.8233 - val_acc: 0.3018
Epoch 7/300
3048/3048 [==============================] - 2s 771us/step - loss: 1.7197 - acc: 0.3468 - val_loss: 1.6899 - val_acc: 0.3

3048/3048 [==============================] - 2s 765us/step - loss: 1.0644 - acc: 0.6280 - val_loss: 1.1071 - val_acc: 0.6194
Epoch 61/300
3048/3048 [==============================] - 2s 760us/step - loss: 1.0480 - acc: 0.6355 - val_loss: 1.1589 - val_acc: 0.5722
Epoch 62/300
3048/3048 [==============================] - 2s 772us/step - loss: 1.0406 - acc: 0.6352 - val_loss: 1.1388 - val_acc: 0.5997
Epoch 63/300
3048/3048 [==============================] - 3s 850us/step - loss: 1.0320 - acc: 0.6463 - val_loss: 1.1852 - val_acc: 0.5787
Epoch 64/300
3048/3048 [==============================] - 2s 763us/step - loss: 1.0273 - acc: 0.6394 - val_loss: 1.1457 - val_acc: 0.5866
Epoch 65/300
3048/3048 [==============================] - 2s 772us/step - loss: 1.0268 - acc: 0.6411 - val_loss: 1.1394 - val_acc: 0.5682
Epoch 66/300
3048/3048 [==============================] - 2s 773us/step - loss: 1.0253 - acc: 0.6411 - val_loss: 1.1936 - val_acc: 0.5840
Epoch 67/300
3048/3048 [=======================

3048/3048 [==============================] - 2s 728us/step - loss: 0.8078 - acc: 0.7175 - val_loss: 1.0242 - val_acc: 0.6286
Epoch 120/300
3048/3048 [==============================] - 2s 716us/step - loss: 0.8081 - acc: 0.7178 - val_loss: 1.1054 - val_acc: 0.6129
Epoch 121/300
3048/3048 [==============================] - 2s 717us/step - loss: 0.8021 - acc: 0.7257 - val_loss: 1.0305 - val_acc: 0.6365
Epoch 122/300
3048/3048 [==============================] - 2s 717us/step - loss: 0.8028 - acc: 0.7159 - val_loss: 1.0358 - val_acc: 0.6247
Epoch 123/300
3048/3048 [==============================] - 2s 723us/step - loss: 0.8069 - acc: 0.7188 - val_loss: 1.0710 - val_acc: 0.6089
Epoch 124/300
3048/3048 [==============================] - 2s 719us/step - loss: 0.8048 - acc: 0.7208 - val_loss: 1.0760 - val_acc: 0.6352
Epoch 125/300
3048/3048 [==============================] - 2s 731us/step - loss: 0.8047 - acc: 0.7185 - val_loss: 0.9953 - val_acc: 0.6378
Epoch 126/300
3048/3048 [================

3048/3048 [==============================] - 2s 718us/step - loss: 0.6683 - acc: 0.7756 - val_loss: 1.0930 - val_acc: 0.6037
Epoch 179/300
3048/3048 [==============================] - 2s 727us/step - loss: 0.6678 - acc: 0.7717 - val_loss: 1.0045 - val_acc: 0.6391
Epoch 180/300
3048/3048 [==============================] - 2s 719us/step - loss: 0.6380 - acc: 0.7936 - val_loss: 1.2229 - val_acc: 0.5853
Epoch 181/300
3048/3048 [==============================] - 2s 734us/step - loss: 0.6676 - acc: 0.7743 - val_loss: 1.0430 - val_acc: 0.6181
Epoch 182/300
3048/3048 [==============================] - 2s 778us/step - loss: 0.6496 - acc: 0.7851 - val_loss: 1.0390 - val_acc: 0.6417
Epoch 183/300
3048/3048 [==============================] - 2s 731us/step - loss: 0.6484 - acc: 0.7851 - val_loss: 1.0707 - val_acc: 0.6102
Epoch 184/300
3048/3048 [==============================] - 2s 716us/step - loss: 0.6667 - acc: 0.7789 - val_loss: 1.1487 - val_acc: 0.5919
Epoch 185/300
3048/3048 [================

Epoch 237/300
3048/3048 [==============================] - 2s 721us/step - loss: 0.5268 - acc: 0.8196 - val_loss: 0.9799 - val_acc: 0.6535
Epoch 238/300
3048/3048 [==============================] - 2s 723us/step - loss: 0.5523 - acc: 0.8156 - val_loss: 1.0408 - val_acc: 0.6365
Epoch 239/300
3048/3048 [==============================] - 2s 720us/step - loss: 0.5361 - acc: 0.8196 - val_loss: 1.0374 - val_acc: 0.6483
Epoch 240/300
3048/3048 [==============================] - 2s 740us/step - loss: 0.5565 - acc: 0.8140 - val_loss: 1.0215 - val_acc: 0.6575
Epoch 241/300
3048/3048 [==============================] - 2s 736us/step - loss: 0.5486 - acc: 0.8192 - val_loss: 0.9822 - val_acc: 0.6811
Epoch 242/300
3048/3048 [==============================] - 2s 724us/step - loss: 0.5467 - acc: 0.8189 - val_loss: 1.0365 - val_acc: 0.6535
Epoch 243/300
3048/3048 [==============================] - 2s 717us/step - loss: 0.5567 - acc: 0.8133 - val_loss: 1.0083 - val_acc: 0.6522
Epoch 244/300
3048/3048 [==

Epoch 296/300
3048/3048 [==============================] - 2s 712us/step - loss: 0.4609 - acc: 0.8524 - val_loss: 1.0007 - val_acc: 0.6811
Epoch 297/300
3048/3048 [==============================] - 2s 718us/step - loss: 0.4730 - acc: 0.8471 - val_loss: 1.0012 - val_acc: 0.6824
Epoch 298/300
3048/3048 [==============================] - 2s 724us/step - loss: 0.4995 - acc: 0.8363 - val_loss: 1.1057 - val_acc: 0.6404
Epoch 299/300
3048/3048 [==============================] - 2s 718us/step - loss: 0.4604 - acc: 0.8497 - val_loss: 1.0032 - val_acc: 0.6562
Epoch 300/300
3048/3048 [==============================] - 2s 718us/step - loss: 0.4773 - acc: 0.8343 - val_loss: 1.0563 - val_acc: 0.6667


In [96]:
test = pd.read_csv(r'data/X_test.csv')
test.drop(['row_id','measurement_number'], axis=1, inplace=True)
test.set_index('series_id',inplace=True)

In [98]:
d = np.array(list(test.groupby('series_id').apply(pd.DataFrame.as_matrix)))
d.shape

c:\users\noam\appdata\local\programs\python\python37\lib\site-packages\pandas\core\groupby\ops.py:942: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return reduction.apply_frame_axis0(sdata, f, names, starts, ends)


(3816, 128, 10)

In [99]:
f = model.predict_classes(d)

In [100]:
f

array([7, 0, 7, ..., 5, 1, 8], dtype=int64)

In [30]:
ohe.inverse_transform(f.reshape(1, -1))

ValueError: Shape of the passed X data is not correct. Expected 9 columns, got 3816.

In [101]:
a = pd.Series([ohe.categories_[0][i] for i in f]).reset_index()
a.columns = ['series_id', 'surface']
a.to_csv(r'submit.csv', index=False)

matrix([[0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])